## Comparing Housing Market "Hotness": Dallas vs Austin

In [7]:
# Import dependencies
import hvplot.pandas
import pandas as pd
import requests
import time
from scipy.stats import linregress
from matplotlib import pyplot as plt
import numpy as np

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

In [9]:
# Load the csv file
austin_dallas_housing_df = pd.read_csv("newdata.csv",encoding='utf-8')
austin_dallas_housing_df.head(10)


,City,State,Month of Period End,Median Sale Price,Median Sale Price MoM,Median Sale Price YoY,Homes Sold,Homes Sold MoM,Homes Sold YoY,New Listings,...,New Listings YoY,Inventory,Inventory MoM,Inventory YoY,Days on Market,Days on Market MoM,Days on Market YoY,Average Sale To List,Average Sale To List MoM,Average Sale To List YoY
0,Austin,Texas,Feb-12,$190K,5.60%,2.20%,"1,403",18.60%,16.60%,"2,569",...,10.40%,"6,941",1.20%,-21.50%,74,0,-18,96.70%,0.40%,0.10%
1,Austin,Texas,Mar-12,$200K,5.10%,8.70%,"2,076",48.00%,18.60%,"3,400",...,-3.00%,"7,394",6.50%,-23.10%,58,-16,-26,97.30%,0.60%,0.90%
2,Austin,Texas,Apr-12,$205K,2.70%,7.90%,"2,120",2.10%,14.80%,"3,379",...,1.40%,"7,698",4.10%,-24.00%,39,-19,-21,97.70%,0.40%,1.00%
3,Austin,Texas,May-12,$210K,2.40%,9.10%,"2,584",21.90%,25.10%,"3,493",...,11.80%,"7,908",2.70%,-22.50%,36,-3,-18,97.60%,0.00%,1.00%
4,Austin,Texas,Jun-12,$214K,2.00%,8.70%,"2,842",10.00%,21.00%,"3,396",...,8.80%,"8,152",3.10%,-20.30%,31,-5,-21,97.80%,0.20%,1.10%
5,Austin,Texas,Jul-12,$210K,-1.90%,10.20%,"2,586",-9.00%,20.30%,"2,939",...,7.20%,"8,068",-1.00%,-18.70%,34,3,-21,97.80%,-0.10%,0.90%
6,Austin,Texas,Aug-12,$209K,-0.50%,8.60%,"2,585",0.00%,17.50%,"2,709",...,15.80%,"7,669",-4.90%,-17.60%,38,4,-23,97.60%,-0.10%,1.00%
7,Austin,Texas,Sep-12,$197K,-6.00%,5.10%,"1,991",-23.00%,10.90%,"2,107",...,1.50%,"7,250",-5.50%,-16.50%,43,5,-20,97.40%,-0.20%,0.90%
8,Austin,Texas,Oct-12,$199K,1.40%,5.40%,"2,157",8.30%,38.70%,"2,234",...,6.90%,"6,773",-6.60%,-18.30%,50,7,-16,97.20%,-0.20%,0.50%
9,Austin,Texas,Nov-12,$199K,0.00%,7.70%,"1,812",-16.00%,23.60%,"1,933",...,10.40%,"6,236",-7.90%,-18.60%,48,-2,-22,97.30%,0.10%,0.60%


In [10]:
#Show data counts
austin_dallas_housing_df.count()

City                         262
State                        262
Month of Period End          262
Median Sale Price            262
Median Sale Price MoM        262
Median Sale Price YoY        262
Homes Sold                   262
Homes Sold MoM               262
Homes Sold YoY               262
New Listings                 262
New Listings MoM             262
New Listings YoY             262
Inventory                    262
Inventory MoM                262
 Inventory YoY               262
Days on Market               262
Days on Market MoM           262
Days on Market YoY           262
Average Sale To List         262
Average Sale To List MoM     262
Average Sale To List YoY     262
dtype: int64

In [13]:
#Trying to separate austin and dallas information
austin_data = austin_dallas_housing_df.loc[austin_dallas_housing_df["City"] == "Austin"]
austin_data

,City,State,Month of Period End,Median Sale Price,Median Sale Price MoM,Median Sale Price YoY,Homes Sold,Homes Sold MoM,Homes Sold YoY,New Listings,...,New Listings YoY,Inventory,Inventory MoM,Inventory YoY,Days on Market,Days on Market MoM,Days on Market YoY,Average Sale To List,Average Sale To List MoM,Average Sale To List YoY
0,Austin,Texas,Feb-12,$190K,5.60%,2.20%,"1,403",18.60%,16.60%,"2,569",...,10.40%,"6,941",1.20%,-21.50%,74,0,-18,96.70%,0.40%,0.10%
1,Austin,Texas,Mar-12,$200K,5.10%,8.70%,"2,076",48.00%,18.60%,"3,400",...,-3.00%,"7,394",6.50%,-23.10%,58,-16,-26,97.30%,0.60%,0.90%
2,Austin,Texas,Apr-12,$205K,2.70%,7.90%,"2,120",2.10%,14.80%,"3,379",...,1.40%,"7,698",4.10%,-24.00%,39,-19,-21,97.70%,0.40%,1.00%
3,Austin,Texas,May-12,$210K,2.40%,9.10%,"2,584",21.90%,25.10%,"3,493",...,11.80%,"7,908",2.70%,-22.50%,36,-3,-18,97.60%,0.00%,1.00%
4,Austin,Texas,Jun-12,$214K,2.00%,8.70%,"2,842",10.00%,21.00%,"3,396",...,8.80%,"8,152",3.10%,-20.30%,31,-5,-21,97.80%,0.20%,1.10%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,Austin,Texas,Aug-22,$500K,-3.80%,5.50%,"2,872",3.10%,-24.00%,"3,672",...,-14.90%,"10,739",-0.20%,77.90%,40,6,10,98.90%,-1.30%,-5.10%
127,Austin,Texas,Sep-22,$475K,-5.00%,5.60%,"2,880",0.30%,-21.10%,"3,528",...,-11.50%,"10,827",0.80%,77.30%,47,7,15,98.30%,-0.60%,-3.90%
128,Austin,Texas,Oct-22,$475K,0.00%,3.70%,"2,235",-22.40%,-29.60%,"2,739",...,-26.10%,"10,291",-5.00%,70.30%,53,6,19,97.30%,-1.00%,-4.40%
129,Austin,Texas,Nov-22,$470K,-1.10%,-0.20%,"1,986",-11.10%,-38.30%,"2,015",...,-30.50%,"9,132",-11.30%,77.30%,61,8,26,96.50%,-0.90%,-4.80%


In [14]:
#Dallas information
dallas_data = austin_dallas_housing_df.loc[austin_dallas_housing_df["City"] == "Dallas"]
dallas_data

,City,State,Month of Period End,Median Sale Price,Median Sale Price MoM,Median Sale Price YoY,Homes Sold,Homes Sold MoM,Homes Sold YoY,New Listings,...,New Listings YoY,Inventory,Inventory MoM,Inventory YoY,Days on Market,Days on Market MoM,Days on Market YoY,Average Sale To List,Average Sale To List MoM,Average Sale To List YoY
131,Dallas,Texas,Feb-12,$160K,4.70%,1.30%,"2,998",20.80%,22.80%,"5,533",...,15.70%,"17,066",4.20%,-22.90%,83,-2,-15,96.40%,0.20%,0.20%
132,Dallas,Texas,Mar-12,$168K,4.70%,8.80%,"3,979",32.70%,12.60%,"6,607",...,-5.30%,"17,862",4.70%,-22.80%,63,-20,-24,96.60%,0.30%,0.60%
133,Dallas,Texas,Apr-12,$169K,0.90%,5.80%,"4,202",5.60%,13.70%,"6,303",...,-4.90%,"18,194",1.90%,-23.60%,49,-14,-23,97.00%,0.40%,0.90%
134,Dallas,Texas,May-12,$178K,5.30%,7.80%,"4,919",17.10%,23.10%,"6,437",...,8.50%,"18,225",0.20%,-22.90%,49,0,-21,97.10%,0.10%,0.80%
135,Dallas,Texas,Jun-12,$180K,1.20%,8.40%,"5,089",3.50%,13.80%,"6,227",...,0.10%,"18,412",1.00%,-22.00%,49,0,-21,97.40%,0.30%,1.10%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,Dallas,Texas,Aug-22,$430K,-1.10%,13.90%,"6,128",6.90%,-11.50%,"6,879",...,-7.10%,"13,765",-2.70%,47.50%,25,6,7,99.70%,-1.80%,-2.70%
258,Dallas,Texas,Sep-22,$420K,-2.30%,12.00%,"5,450",-11.10%,-17.90%,"6,459",...,-4.70%,"13,887",0.90%,54.60%,32,7,11,98.70%,-1.00%,-2.80%
259,Dallas,Texas,Oct-22,$420K,0.00%,11.10%,"4,573",-16.10%,-26.30%,"5,564",...,-9.30%,"13,714",-1.20%,67.80%,33,1,11,98.50%,-0.20%,-2.80%
260,Dallas,Texas,Nov-22,$410K,-2.40%,8.30%,"4,064",-11.10%,-30.50%,"4,439",...,-14.10%,"12,697",-7.40%,83.90%,37,4,16,97.90%,-0.60%,-3.60%


In [16]:
#Check data types for each column
austin_data.dtypes

City                         object
State                        object
Month of Period End          object
Median Sale Price            object
Median Sale Price MoM        object
Median Sale Price YoY        object
Homes Sold                   object
Homes Sold MoM               object
Homes Sold YoY               object
New Listings                 object
New Listings MoM             object
New Listings YoY             object
Inventory                    object
Inventory MoM                object
 Inventory YoY               object
Days on Market                int64
Days on Market MoM            int64
Days on Market YoY            int64
Average Sale To List         object
Average Sale To List MoM     object
Average Sale To List YoY     object
dtype: object